<a href="https://colab.research.google.com/github/DebajyotiBindu/AI-text-Generator/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets accelerate -U

import os
import torch
from datasets import load_dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

input_file = "/content/data.txt"
output_dir = "./model_gpt2"

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)

def group_texts(examples):

    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    block_size = 128
    total_length = (total_length // block_size) * block_size

    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

dataset = load_dataset("text", data_files={"train": input_file})

tokenized_dataset = dataset.map(
    lambda x: tokenizer(x["text"]),
    batched=True,
    remove_columns=["text"]
).filter(lambda x: len(x["input_ids"]) > 0)

lm_dataset = tokenized_dataset.map(group_texts, batched=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_total_limit=1,
    logging_steps=50,
    prediction_loss_only=True,
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

trainer.train()

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
!zip -r sherlock_gpt2_model.zip ./model_gpt2



In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
!zip -r sherlock_gpt2_model.zip ./model_gpt2
